In [1]:
import numpy as np

In [61]:
class LinearRegression_manual:
    def __init__(self, learning_rate, epochs):
        self.learning_rate = learning_rate
        self.epochs = epochs
    
    def fit(self, x, y):
        self.m, self.n = x.shape
        self.y = y
        self.x = np.c_[np.ones((self.m, 1)), x]
        self.theta = np.zeros((self.n+1,1))
        for epoch in range(self.epochs):
            if(epoch%10000==0):print(f"{epoch=}")
            gradients = self.x.T.dot(self.x.dot(self.theta) - self.y) / self.m
            self.theta -= self.learning_rate * gradients
    
    def predict(self, x):
        x = np.c_[np.ones((x.shape[0], 1)), x]
        return x.dot(self.theta)

    def mean_squared_error(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)

In [72]:
if __name__ == "__main__":
    X = np.array([[1], [2], [4], [3], [5]])
    y = np.array([[1.5], [3.5], [6.0], [4.5], [9.0]])

    lr = LinearRegression_manual(learning_rate=0.001, epochs=100000)
    lr.fit(X, y)
    
    training_predictions = lr.predict(X)
    training_error = lr.mean_squared_error(y, training_predictions)
    print("Training Mean Squared Error:", training_error)

    X_new = np.array([[1.5], [2.5], [3.5]])
    predictions = lr.predict(X_new)
    print("Predictions:", predictions)
    y_new_true = np.array([[2.3], [4.0], [5.5]])  # Example true values
    prediction_error = lr.mean_squared_error(y_new_true, predictions)
    print("Prediction Mean Squared Error:", prediction_error)

epoch=0
epoch=10000
epoch=20000
epoch=30000
epoch=40000
epoch=50000
epoch=60000
epoch=70000
epoch=80000
epoch=90000
Training Mean Squared Error: 0.21499999999999994
Predictions: [[2.27500002]
 [4.02500001]
 [5.775     ]]
Prediction Mean Squared Error: 0.025625000034740594
